In [ ]:
import os
import requests
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

API_KEY = os.environ.get("ALPHAVANTAGE_API_KEY")
if not API_KEY:
    raise RuntimeError("Set ALPHAVANTAGE_API_KEY environment variable before running this cell")


In [ ]:
def generate_time_windows(start_date: str, end_date: str, window_days: int = 5):
    fmt = "%Y%m%dT%H%M"
    start = datetime.strptime(start_date, fmt)
    end = datetime.strptime(end_date, fmt)
    windows = []
    while start < end:
        window_end = start + timedelta(days=window_days)
        if window_end > end:
            window_end = end
        windows.append((start.strftime(fmt), window_end.strftime(fmt)))
        start = window_end
    return windows


def fetch_nvidia_news_alpha(limit: int = 1000, time_from: str = None, time_to: str = None):
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": "NVDA",
        "limit": str(limit),
        "apikey": API_KEY,
        "time_from": time_from,
        "time_to": time_to,
        "sort": "EARLIEST",
    }
    url = "https://www.alphavantage.co/query"
    resp = requests.get(url, params=params, timeout=30)
    resp.raise_for_status()
    news_json = resp.json()
    return news_json


# Generate sliding windows
windows_2020 = generate_time_windows("20200101T0000", "20201231T2359", window_days=5)
windows_2021 = generate_time_windows("20210101T0000", "20211231T2359", window_days=5)
windows_2022 = generate_time_windows("20220101T0000", "20221231T2359", window_days=5)
windows_2023 = generate_time_windows("20230101T0000", "20231231T2359", window_days=5)
windows_2024 = generate_time_windows("20240101T0000", "20241231T2359", window_days=5)
windows_2025 = generate_time_windows("20250101T0000", "20250630T2359", window_days=5)
print(len(windows_2020))
print(len(windows_2021))
print(len(windows_2022))
print(len(windows_2023))
print(len(windows_2024))
print(len(windows_2025))


In [ ]:
REQUESTS_PER_MINUTE = 75
SLEEP_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE


def fetch_all_news_windows(windows, year: int):
    all_news = []
    for time_from, time_to in windows:
        print(f"Fetching news from {time_from} to {time_to}...")
        news = fetch_nvidia_news_alpha(time_from=time_from, time_to=time_to)
        all_news.append(news)
        time.sleep(SLEEP_BETWEEN_REQUESTS)
    os.makedirs("news", exist_ok=True)

    with open(f"news/nvidia_news_{year}.json", "w") as f:
        json.dump(all_news, f, indent=2)


fetch_all_news_windows(windows_2020, 2020)
fetch_all_news_windows(windows_2021, 2021)
fetch_all_news_windows(windows_2022, 2022)
fetch_all_news_windows(windows_2023, 2023)
fetch_all_news_windows(windows_2024, 2024)
fetch_all_news_windows(windows_2025, 2025)

In [4]:
import pandas as pd


def extract_nvda_info(feed_list):
    rows = []
    for item in feed_list:
        # Find NVDA ticker sentiment
        nvda_info = next((t for t in item.get("ticker_sentiment", []) if t.get("ticker") == "NVDA"), None)
        if nvda_info:
            rows.append(
                {
                    "title": item.get("title"),
                    "url": item.get("url"),
                    "time_published": item.get("time_published"),
                    "authors": item.get("authors"),
                    "summary": item.get("summary"),
                    "banner_image": item.get("banner_image"),
                    "source": item.get("source"),
                    "category_within_source": item.get("category_within_source"),
                    "source_domain": item.get("source_domain"),
                    "overall_sentiment_score": item.get("overall_sentiment_score"),
                    "overall_sentiment_label": item.get("overall_sentiment_label"),
                    "nvda_sentiment_score": nvda_info.get("ticker_sentiment_score"),
                    "nvda_sentiment_label": nvda_info.get("ticker_sentiment_label"),
                    "nvda_relevance_score": nvda_info.get("relevance_score"),
                }
            )
    return pd.DataFrame(rows)


In [10]:
# Fetch and save NVDA news
all_news = []
for year in range(2020, 2026):
    with open(f"news/nvidia_news_{year}.json", "r") as f:
        year_news = json.load(f)
        all_news.extend(year_news)

df_chunks = []
for news_chunk in all_news:
    feed_list = news_chunk.get("feed", [])
    df_chunk = extract_nvda_info(feed_list)
    df_chunks.append(df_chunk)

df_nvda_news = pd.concat(df_chunks, ignore_index=True)
df_nvda_news

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,nvda_sentiment_score,nvda_sentiment_label,nvda_relevance_score
0,MarketingPulse and eTailingPulse to shed light...,https://www.scmp.com/presented/business/topics...,20220303T065619,[Advertising partner],[Sponsored Article]\r\rThe Covid-19 pandemic h...,https://cdn.i-scmp.com/sites/default/files/sty...,South China Morning Post,Business,www.scmp.com,0.010325,Neutral,0.007051,Neutral,0.045456
1,20 Stocks Billionaires Are Selling,https://www.kiplinger.com/investing/stocks/604...,20220303T210336,[Will Ashworth],A lot of money continues to flow into hedge fu...,https://mediacloud.kiplinger.com/image/private...,Kiplinger,n/a,www.kiplinger.com,-0.009081,Neutral,0.025975,Neutral,0.060219
2,The Top 5 Investment Plays for Blockchain,https://www.zacks.com/stock/news/1877770/the-t...,20220304T172200,[Zacks Investment Research],There are plenty of ways to invest in blockcha...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,0.125201,Neutral,0.236167,Somewhat-Bullish,0.02519
3,Nvidia Faces an Unusual Demand That Threatens ...,https://www.thestreet.com/investing/nvidia-fac...,20220304T193600,[Riley Gutiérrez McDermid],The chip giant was recently the victim of a hack.,https://www.thestreet.com/.image/t_share/MTg1O...,The Street,n/a,www.thestreet.com,-0.311832,Somewhat-Bearish,-0.361876,Bearish,0.609839
4,"Amid Russia's war, America Inc reckons with th...",https://www.economist.com/business/2022/03/06/...,20220306T000000,[],Multinational companies grapple with a fractur...,https://www.economist.com/img/b/608/662/90/sit...,The Economist,Business,www.economist.com,-0.365286,Bearish,-0.482104,Bearish,0.022664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44697,Prediction: These Will Be the Two Best-Perform...,https://www.fool.com/investing/2025/06/30/pred...,20250630T200000,[Keith Noonan and Jennifer Saibil],"The ""Magnificent Seven"" is a group of technolo...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,0.358923,Bullish,0.043446,Neutral,0.043264
44698,IREN Achieves Mid-Year Target of 50 EH/s - IRE...,https://www.benzinga.com/pressreleases/25/06/g...,20250630T212841,[Globe Newswire],"SYDNEY, June 30, 2025 ( GLOBE NEWSWIRE ) -- IR...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,General,www.benzinga.com,0.129430,Neutral,-0.17714,Somewhat-Bearish,0.015675
44699,Markets Close Rollercoaster Q2 on Top,https://www.zacks.com/stock/news/2558429/marke...,20250630T215800,[Mark Vickery],"After starting calendar Q2 with a huge ""Libera...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,-0.004783,Neutral,0.189457,Somewhat-Bullish,0.15697
44700,How to Find Attractively Priced Growth Stocks,https://www.zacks.com/stock/news/2558382/how-t...,20250630T215800,[Neena Mishra],We highlight two popular GARP ETFs.,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,0.446589,Bullish,0.35385,Bullish,0.451494


In [11]:
df_nvda_news.index = pd.to_datetime(df_nvda_news["time_published"], format="%Y%m%dT%H%M%S")
df_nvda_news = df_nvda_news.drop(columns=["time_published"])
df_nvda_news = df_nvda_news.sort_index()
df_nvda_news

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,nvda_sentiment_score,nvda_sentiment_label,nvda_relevance_score
time_published,,,,,,,,,,,,,
2022-03-03 06:56:19,MarketingPulse and eTailingPulse to shed light...,https://www.scmp.com/presented/business/topics...,[Advertising partner],[Sponsored Article]\r\rThe Covid-19 pandemic h...,https://cdn.i-scmp.com/sites/default/files/sty...,South China Morning Post,Business,www.scmp.com,0.010325,Neutral,0.007051,Neutral,0.045456
2022-03-03 21:03:36,20 Stocks Billionaires Are Selling,https://www.kiplinger.com/investing/stocks/604...,[Will Ashworth],A lot of money continues to flow into hedge fu...,https://mediacloud.kiplinger.com/image/private...,Kiplinger,n/a,www.kiplinger.com,-0.009081,Neutral,0.025975,Neutral,0.060219
2022-03-04 17:22:00,The Top 5 Investment Plays for Blockchain,https://www.zacks.com/stock/news/1877770/the-t...,[Zacks Investment Research],There are plenty of ways to invest in blockcha...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,0.125201,Neutral,0.236167,Somewhat-Bullish,0.02519
2022-03-04 19:36:00,Nvidia Faces an Unusual Demand That Threatens ...,https://www.thestreet.com/investing/nvidia-fac...,[Riley Gutiérrez McDermid],The chip giant was recently the victim of a hack.,https://www.thestreet.com/.image/t_share/MTg1O...,The Street,n/a,www.thestreet.com,-0.311832,Somewhat-Bearish,-0.361876,Bearish,0.609839
2022-03-06 00:00:00,"Amid Russia's war, America Inc reckons with th...",https://www.economist.com/business/2022/03/06/...,[],Multinational companies grapple with a fractur...,https://www.economist.com/img/b/608/662/90/sit...,The Economist,Business,www.economist.com,-0.365286,Bearish,-0.482104,Bearish,0.022664
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30 20:00:00,Prediction: These Will Be the Two Best-Perform...,https://www.fool.com/investing/2025/06/30/pred...,[Keith Noonan and Jennifer Saibil],"The ""Magnificent Seven"" is a group of technolo...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,0.358923,Bullish,0.043446,Neutral,0.043264
2025-06-30 21:28:41,IREN Achieves Mid-Year Target of 50 EH/s - IRE...,https://www.benzinga.com/pressreleases/25/06/g...,[Globe Newswire],"SYDNEY, June 30, 2025 ( GLOBE NEWSWIRE ) -- IR...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,General,www.benzinga.com,0.129430,Neutral,-0.17714,Somewhat-Bearish,0.015675
2025-06-30 21:58:00,Markets Close Rollercoaster Q2 on Top,https://www.zacks.com/stock/news/2558429/marke...,[Mark Vickery],"After starting calendar Q2 with a huge ""Libera...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,-0.004783,Neutral,0.189457,Somewhat-Bullish,0.15697


In [12]:
df_nvda_news.to_parquet("news/nvidia_news.parquet")

In [15]:
pd.read_parquet("news/nvidia_news.parquet")

,title,url,authors,summary,banner_image,source,category_within_source,source_domain,overall_sentiment_score,overall_sentiment_label,nvda_sentiment_score,nvda_sentiment_label,nvda_relevance_score
time_published,,,,,,,,,,,,,
2022-03-03 06:56:19,MarketingPulse and eTailingPulse to shed light...,https://www.scmp.com/presented/business/topics...,[Advertising partner],[Sponsored Article]\r\rThe Covid-19 pandemic h...,https://cdn.i-scmp.com/sites/default/files/sty...,South China Morning Post,Business,www.scmp.com,0.010325,Neutral,0.007051,Neutral,0.045456
2022-03-03 21:03:36,20 Stocks Billionaires Are Selling,https://www.kiplinger.com/investing/stocks/604...,[Will Ashworth],A lot of money continues to flow into hedge fu...,https://mediacloud.kiplinger.com/image/private...,Kiplinger,n/a,www.kiplinger.com,-0.009081,Neutral,0.025975,Neutral,0.060219
2022-03-04 17:22:00,The Top 5 Investment Plays for Blockchain,https://www.zacks.com/stock/news/1877770/the-t...,[Zacks Investment Research],There are plenty of ways to invest in blockcha...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,0.125201,Neutral,0.236167,Somewhat-Bullish,0.02519
2022-03-04 19:36:00,Nvidia Faces an Unusual Demand That Threatens ...,https://www.thestreet.com/investing/nvidia-fac...,[Riley Gutiérrez McDermid],The chip giant was recently the victim of a hack.,https://www.thestreet.com/.image/t_share/MTg1O...,The Street,n/a,www.thestreet.com,-0.311832,Somewhat-Bearish,-0.361876,Bearish,0.609839
2022-03-06 00:00:00,"Amid Russia's war, America Inc reckons with th...",https://www.economist.com/business/2022/03/06/...,[],Multinational companies grapple with a fractur...,https://www.economist.com/img/b/608/662/90/sit...,The Economist,Business,www.economist.com,-0.365286,Bearish,-0.482104,Bearish,0.022664
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30 20:00:00,Prediction: These Will Be the Two Best-Perform...,https://www.fool.com/investing/2025/06/30/pred...,[Keith Noonan and Jennifer Saibil],"The ""Magnificent Seven"" is a group of technolo...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,0.358923,Bullish,0.043446,Neutral,0.043264
2025-06-30 21:28:41,IREN Achieves Mid-Year Target of 50 EH/s - IRE...,https://www.benzinga.com/pressreleases/25/06/g...,[Globe Newswire],"SYDNEY, June 30, 2025 ( GLOBE NEWSWIRE ) -- IR...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,General,www.benzinga.com,0.129430,Neutral,-0.17714,Somewhat-Bearish,0.015675
2025-06-30 21:58:00,Markets Close Rollercoaster Q2 on Top,https://www.zacks.com/stock/news/2558429/marke...,[Mark Vickery],"After starting calendar Q2 with a huge ""Libera...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,-0.004783,Neutral,0.189457,Somewhat-Bullish,0.15697
